
## Introduction
Welcome to "***AutoScout Data Analysis Project***". This is the capstone project of ***Data Analysis*** Module. **Auto Scout** data which using for this project, scraped from the on-line car trading company in 2019, contains many features of 9 different car models. In this project, you will have the opportunity to apply many commonly used algorithms for Data Cleaning and Exploratory Data Analysis by using many Python libraries such as Numpy, Pandas, Matplotlib, Seaborn, Scipy you will analyze clean dataset.

The project consists of 3 parts:
* First part is related with 'data cleaning'. It deals with Incorrect Headers, Incorrect Format, Anomalies, Dropping useless columns.
* Second part is related with 'filling data'. It deals with Missing Values. Categorical to numeric transformation is done.
* Third part is related with 'handling outliers of data' via Visualisation libraries. Some insights are extracted.

# PART- 3 `( Handling Outliers )`

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

%matplotlib inline
#%matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100pd.set_option('display.max_columns', 500)

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 150
pd.options.display.max_columns = 100

C:\Users\Toshiba\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Toshiba\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Toshiba\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
df0 = pd.read_csv('autoscoutpart_2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'autoscoutpart_2.csv'

In [ ]:
df = df0.copy()

## functions to fill the missing values

In [ ]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill_prop(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
df.shape

In [ ]:
df.head(1).T

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df['cons_city_l/100_km'].skew()

In [ ]:
df['cons_comb_l/100_km'].skew()

In [ ]:
df['cons_country_l/100_km'].skew()

In [ ]:
df.describe(include=object).T

In [ ]:
num = df.drop(['price'], axis=1).select_dtypes('number').columns

In [ ]:
df[num]

In [ ]:
sns.pairplot(df[num], size = 2.5)
plt.show();

In [ ]:
sns.heatmap(df.corr(), annot=True);

In [ ]:
df.columns

In [ ]:
#df.drop(['cons_city_l/100_km', 'cons_country_l/100_km'], axis=1, inplace=True)

In [ ]:
df.head().T

In [ ]:
df.shape

In [ ]:
sns.heatmap(df.corr(), annot=True);

In [ ]:
df.shape

## Dummy Operation

In [ ]:
#df.to_csv("autoscoutpart_3_before_dummy.csv", index=False)

In [ ]:
cat = df.select_dtypes('object').columns
cat

In [ ]:
#df = df.join(df["comfort_convenience"].str.get_dummies(sep = ",").add_prefix("coco_"))
#df = df.join(df["entertainment_media"].str.get_dummies(sep = ",").add_prefix("entmedia_"))
#df = df.join(df["extras"].str.get_dummies(sep = ",").add_prefix("extrs_"))
#df = df.join(df["safety_security"].str.get_dummies(sep = ",").add_prefix("safesec_"))

In [ ]:
#df.drop(["comfort_convenience","entertainment_media","extras","safety_security"], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
#df_final = pd.get_dummies(df)

In [ ]:
df_final.shape

In [ ]:
#df_final.head()

In [149]:
#df_final.columns= df_final.columns.str.lower().str.replace('&', '_').str.replace(' ', '_')

In [150]:
df_final.head()

,price,km,nr._of_doors,nr._of_seats,cylinders,co2_emission,emission_label,gears,age,hp_kw,displacement_cc,weight_kg,cons_comb_l/100_km,coco_air_conditioning,coco_air_suspension,coco_armrest,coco_automatic_climate_control,coco_auxiliary_heating,coco_cruise_control,coco_electric_starter,coco_electric_tailgate,coco_electrical_side_mirrors,coco_electrically_adjustable_seats,coco_electrically_heated_windshield,coco_heads-up_display,coco_heated_steering_wheel,coco_hill_holder,coco_keyless_central_door_lock,coco_leather_seats,coco_leather_steering_wheel,coco_light_sensor,coco_lumbar_support,coco_massage_seats,coco_multi-function_steering_wheel,coco_navigation_system,coco_panorama_roof,coco_park_distance_control,coco_parking_assist_system_camera,coco_parking_assist_system_self-steering,coco_parking_assist_system_sensors_front,coco_parking_assist_system_sensors_rear,coco_power_windows,coco_rain_sensor,coco_seat_heating,coco_seat_ventilation,coco_split_rear_seats,coco_start-stop_system,coco_sunroof,coco_tinted_windows,coco_wind_deflector,...,body_color_black,body_color_blue,body_color_bronze,body_color_brown,body_color_gold,body_color_gray,body_color_green,body_color_grey,body_color_orange,body_color_red,body_color_silver,body_color_violet,body_color_white,body_color_yellow,paint_type_metallic,paint_type_perl_effect,paint_type_uni/basic,gearing_type_automatic,gearing_type_manual,gearing_type_semi-automatic,drive_chain_4wd,drive_chain_front,fuel_benzin,fuel_diesel,fuel_hybrid,fuel_lpg,fuel_others,emission_class_euro_4,emission_class_euro_5,emission_class_euro_6,vehicle_condition_new,vehicle_condition_used,warranty_month_no_warranty,warranty_month_yes_warranty,upholstery_type_cloth,upholstery_type_full_leather,upholstery_type_other,upholstery_type_part_leather,upholstery_type_velour,upholstery_type_alcantara,upholstery_color_beige,upholstery_color_black,upholstery_color_blue,upholstery_color_brown,upholstery_color_grey,upholstery_color_orange,upholstery_color_other,upholstery_color_red,upholstery_color_white,upholstery_color_yellow
0,15770.000,56013.000,5.000,5.000,3.000,99.000,4.000,7.000,3.000,66.000,1422.000,1220.000,3.800,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,14500.000,80000.000,3.000,4.000,4.000,129.000,4.000,7.000,2.000,141.000,1798.000,1255.000,5.600,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,14640.000,83450.000,4.000,4.000,4.000,99.000,4.000,7.000,3.000,85.000,1598.000,1275.000,3.800,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,14500.000,73000.000,3.000,4.000,3.000,99.000,4.000,6.000,3.000,66.000,1422.000,1195.000,3.800,0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,16790.000,16200.000,5.000,5.000,3.000,109.000,4.000,7.000,3.000,66.000,1422.000,1195.000,4.100,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [146]:
df_final.info

<bound method DataFrame.info of           price        km  nr._of_doors  nr._of_seats  cylinders  \
0     15770.000 56013.000         5.000         5.000      3.000   
1     14500.000 80000.000         3.000         4.000      4.000   
2     14640.000 83450.000         4.000         4.000      4.000   
3     14500.000 73000.000         3.000         4.000      3.000   
4     16790.000 16200.000         5.000         5.000      3.000   
...         ...       ...           ...           ...        ...   
15914 39950.000  1652.839         5.000         5.000      4.000   
15915 39885.000  9900.000         5.000         5.000      4.000   
15916 39875.000    15.000         5.000         7.000      4.000   
15917 39700.000    10.000         5.000         7.000      4.000   
15918 40999.000  1652.839         5.000         5.000      4.000   

       co2_emission  emission_label  gears   age   hp_kw  displacement_cc  \
0            99.000           4.000  7.000 3.000  66.000         1422.000 

In [152]:
df_final.to_csv("autoscoutpart_3_dummy.csv", index=False)

In [154]:
df1 = pd.read_csv('autoscoutpart_3_dummy.csv')

In [155]:
df1.head()

,price,km,nr._of_doors,nr._of_seats,cylinders,co2_emission,emission_label,gears,age,hp_kw,displacement_cc,weight_kg,cons_comb_l/100_km,coco_air_conditioning,coco_air_suspension,coco_armrest,coco_automatic_climate_control,coco_auxiliary_heating,coco_cruise_control,coco_electric_starter,coco_electric_tailgate,coco_electrical_side_mirrors,coco_electrically_adjustable_seats,coco_electrically_heated_windshield,coco_heads-up_display,coco_heated_steering_wheel,coco_hill_holder,coco_keyless_central_door_lock,coco_leather_seats,coco_leather_steering_wheel,coco_light_sensor,coco_lumbar_support,coco_massage_seats,coco_multi-function_steering_wheel,coco_navigation_system,coco_panorama_roof,coco_park_distance_control,coco_parking_assist_system_camera,coco_parking_assist_system_self-steering,coco_parking_assist_system_sensors_front,coco_parking_assist_system_sensors_rear,coco_power_windows,coco_rain_sensor,coco_seat_heating,coco_seat_ventilation,coco_split_rear_seats,coco_start-stop_system,coco_sunroof,coco_tinted_windows,coco_wind_deflector,...,body_color_black,body_color_blue,body_color_bronze,body_color_brown,body_color_gold,body_color_gray,body_color_green,body_color_grey,body_color_orange,body_color_red,body_color_silver,body_color_violet,body_color_white,body_color_yellow,paint_type_metallic,paint_type_perl_effect,paint_type_uni/basic,gearing_type_automatic,gearing_type_manual,gearing_type_semi-automatic,drive_chain_4wd,drive_chain_front,fuel_benzin,fuel_diesel,fuel_hybrid,fuel_lpg,fuel_others,emission_class_euro_4,emission_class_euro_5,emission_class_euro_6,vehicle_condition_new,vehicle_condition_used,warranty_month_no_warranty,warranty_month_yes_warranty,upholstery_type_cloth,upholstery_type_full_leather,upholstery_type_other,upholstery_type_part_leather,upholstery_type_velour,upholstery_type_alcantara,upholstery_color_beige,upholstery_color_black,upholstery_color_blue,upholstery_color_brown,upholstery_color_grey,upholstery_color_orange,upholstery_color_other,upholstery_color_red,upholstery_color_white,upholstery_color_yellow
0,15770.000,56013.000,5.000,5.000,3.000,99.000,4.000,7.000,3.000,66.000,1422.000,1220.000,3.800,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,14500.000,80000.000,3.000,4.000,4.000,129.000,4.000,7.000,2.000,141.000,1798.000,1255.000,5.600,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,14640.000,83450.000,4.000,4.000,4.000,99.000,4.000,7.000,3.000,85.000,1598.000,1275.000,3.800,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,14500.000,73000.000,3.000,4.000,3.000,99.000,4.000,6.000,3.000,66.000,1422.000,1195.000,3.800,0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,16790.000,16200.000,5.000,5.000,3.000,109.000,4.000,7.000,3.000,66.000,1422.000,1195.000,4.100,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
